# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
cities_df = pd.read_csv('Resources/cities.csv')

cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hobart,-42.8794,147.3294,54.01,96,90,1.99,AU,1623942850
1,1,Narsaq,60.9167,-46.0500,53.82,71,56,6.46,GL,1623942850
2,2,Cape Town,-33.9258,18.4232,64.09,82,94,1.01,ZA,1623942673
3,3,Chui,-33.6971,-53.4616,53.35,79,100,10.87,UY,1623942851
4,4,Charleville,-26.4000,146.2500,42.87,87,18,0.00,AU,1623942851


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [56]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"].astype(float)

 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
cities_cleaned_df = cities_df.loc[(cities_df['Max Temp'] >= 75) & (cities_df['Max Temp']<= 90) & (cities_df['Wind Speed'] >= 10) & (cities_df['Cloudiness'] >= 30)]
cities_cleaned_df = cities_cleaned_df[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date']]
cities_cleaned_df.reset_index(drop=True, inplace=True)
cities_cleaned_df.dropna(inplace = True) 
cities_cleaned_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mahuva,21.0914,71.7622,85.50,71,97,19.30,IN,1623942856
1,Cabra,37.4725,-4.4421,78.39,43,97,11.16,ES,1623942860
2,Porbandar,21.6422,69.6093,85.71,74,39,16.06,IN,1623942863
3,Bathsheba,13.2167,-59.5167,84.81,61,40,26.46,BB,1623942870
4,Fare,-16.7000,-151.0167,80.42,71,77,23.38,PF,1623942871


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = cities_cleaned_df
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mahuva,21.0914,71.7622,85.50,71,97,19.30,IN,1623942856,
1,Cabra,37.4725,-4.4421,78.39,43,97,11.16,ES,1623942860,
2,Porbandar,21.6422,69.6093,85.71,74,39,16.06,IN,1623942863,
3,Bathsheba,13.2167,-59.5167,84.81,61,40,26.46,BB,1623942870,
4,Fare,-16.7000,-151.0167,80.42,71,77,23.38,PF,1623942871,


In [74]:
target_type = "lodging"
keyword = "hotel"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    locations = f'{lat},{lng}'
    
    city = row["City"]
    params = {
        "location": locations,
        "type": target_type,
        "keyword": keyword,
        "radius": 5000,
        "key": g_key,
        }

    # assemble url and make API request
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Retrieving Results for closest hotel to {city}.")      
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
              
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
print("Search Complete")

Retrieving Results for closest hotel to Mahuva.
------------
Retrieving Results for closest hotel to Cabra.
------------
Retrieving Results for closest hotel to Porbandar.
------------
Retrieving Results for closest hotel to Bathsheba.
------------
Retrieving Results for closest hotel to Fare.
------------
Retrieving Results for closest hotel to Butaritari.
Missing field/result... skipping.
------------
Retrieving Results for closest hotel to Oranjestad.
------------
Retrieving Results for closest hotel to The Valley.
------------
Retrieving Results for closest hotel to Puerto del Rosario.
------------
Retrieving Results for closest hotel to Dondaicha.
------------
Retrieving Results for closest hotel to Port-Gentil.
------------
Retrieving Results for closest hotel to Rock Sound.
------------
Retrieving Results for closest hotel to Itarema.
------------
Retrieving Results for closest hotel to Cockburn Town.
------------
Retrieving Results for closest hotel to Denpasar.
------------
Re

In [75]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mahuva,21.0914,71.7622,85.50,71,97,19.30,IN,1623942856,Hotel Heaven
1,Cabra,37.4725,-4.4421,78.39,43,97,11.16,ES,1623942860,MS Fuente Las Piedras
2,Porbandar,21.6422,69.6093,85.71,74,39,16.06,IN,1623942863,Lords Inn Porbandar
3,Bathsheba,13.2167,-59.5167,84.81,61,40,26.46,BB,1623942870,Atlantis Hotel
4,Fare,-16.7000,-151.0167,80.42,71,77,23.38,PF,1623942871,Maitai Lapita Village Huahine


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [101]:
text_dict = pd.DataFrame({
    "Name": hotel_df["Hotel Name"],
    "City": hotel_df["City"],
    "Country":hotel_df["Country"]})
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mahuva,21.0914,71.7622,85.50,71,97,19.30,IN,1623942856,Hotel Heaven
1,Cabra,37.4725,-4.4421,78.39,43,97,11.16,ES,1623942860,MS Fuente Las Piedras
2,Porbandar,21.6422,69.6093,85.71,74,39,16.06,IN,1623942863,Lords Inn Porbandar
3,Bathsheba,13.2167,-59.5167,84.81,61,40,26.46,BB,1623942870,Atlantis Hotel
4,Fare,-16.7000,-151.0167,80.42,71,77,23.38,PF,1623942871,Maitai Lapita Village Huahine


In [104]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
 
# Add layer to map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))